In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

def load_splits(train_path="train.csv", val_path="validation.csv", test_path="test.csv"):
    """Loads train, validation, and test splits from CSV files."""
    train = pd.read_csv(train_path)
    val = pd.read_csv(val_path)
    test = pd.read_csv(test_path)
    return train, val, test

def preprocess_text(train, val, test):
    """Converts text data to TF-IDF vectors."""
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(train['message'])
    X_val = vectorizer.transform(val['message'])
    X_test = vectorizer.transform(test['message'])

    y_train = train['label']
    y_val = val['label']
    y_test = test['label']

    return X_train, y_train, X_val, y_val, X_test, y_test, vectorizer

def fit_model(model, X_train, y_train):
    """Fits a model on the training data."""
    model.fit(X_train, y_train)
    return model

def score_model(model, X, y):
    """Scores the model on given data."""
    return model.score(X, y)

def evaluate_model(model, X, y):
    """Evaluates the model using accuracy and classification report."""
    y_pred = model.predict(X)
    acc = accuracy_score(y, y_pred)
    report = classification_report(y, y_pred)
    return acc, report

def validate_model(model, X_train, y_train, X_val, y_val):
    """Fits on train, scores on train and validation, evaluates on train and validation."""
    model = fit_model(model, X_train, y_train)
    
    print("Train Score:", score_model(model, X_train, y_train))
    print("Validation Score:", score_model(model, X_val, y_val))
    
    train_acc, train_report = evaluate_model(model, X_train, y_train)
    val_acc, val_report = evaluate_model(model, X_val, y_val)
    
    print("\nTrain Evaluation:\n", train_report)
    print("\nValidation Evaluation:\n", val_report)
    
    return model

def tune_hyperparameters(X_train, y_train, X_val, y_val):
    """Fine-tunes hyperparameters using train and validation data."""
    param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    grid = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy')
    grid.fit(X_train, y_train)
    
    best_model = grid.best_estimator_
    print("Best Hyperparameters:", grid.best_params_)
    
    return validate_model(best_model, X_train, y_train, X_val, y_val)

def benchmark_models(X_train, y_train, X_val, y_val, X_test, y_test):
    """Trains and evaluates three benchmark models, selecting the best one."""
    models = {
        "Logistic Regression": LogisticRegression(),
        "Naive Bayes": MultinomialNB(),
        "SVM": SVC()
    }
    
    best_model = None
    best_score = 0

    for name, model in models.items():
        print(f"\nTraining {name}...")
        model = validate_model(model, X_train, y_train, X_val, y_val)
        test_acc, _ = evaluate_model(model, X_test, y_test)
        
        print(f"Test Accuracy for {name}: {test_acc}")
        
        if test_acc > best_score:
            best_score = test_acc
            best_model = model
    
    print("\nBest Model:", best_model)
    return best_model

